In [1]:
YDSHPERE = False

In [2]:
if YDSHPERE:
    %pip install efficientnet_pytorch
else:
    !pwd

/u01/mrorange/seti


In [3]:
#!g1.1
import os
import gc
import sys
import cv2
import glob
import time
import json
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torch.nn as nn
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn import metrics
from tqdm.notebook import tqdm
from efficientnet_pytorch import model as enet
#warnings.filterwarnings('ignore', category=UserWarning) 
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('GPU is available')
else:
    DEVICE = torch.device('cpu')
    print('CPU is used')

GPU is available


In [4]:
VER = 'v2'
DEBUG = False
PARAMS = {
    'version': VER,
    'folds': 5,
    'folds_train': None,
    'batch_size': 24,
    'workers': 8,
    'epochs': 4 if DEBUG else 100,
    'patience': 8,
    'warmup': False,
    'backbone': 'efficientnet-b3', # 'efficientnet-bX' or 'resnext'
    'dropout': .4,
    'seed': 2020,
    'lr': .001,
    'comments': ''
}
DATA_PATH = './data'
IMGS_PATH = f'{DATA_PATH}/train'
MDLS_PATH = f'./models_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/params.json', 'w') as file:
    json.dump(PARAMS, file)

def seed_all(seed=0):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state    

random_state = seed_all(PARAMS['seed'])

start_time = time.time()

In [5]:
if DEBUG:
    df = pd.read_csv(f'{DATA_PATH}/train_labels.csv').sample(100).reset_index(drop=True)
else:
    df = pd.read_csv(f'{DATA_PATH}/train_labels.csv')
print (df.shape)
df['img_path'] = df['id'].apply(lambda x: f'{IMGS_PATH}/{x[0]}/{x}.npy')
df.head()

(50165, 2)


,id,target,img_path
0,00034abb3629,0,./data/train/0/00034abb3629.npy
1,0004300a0b9b,0,./data/train/0/0004300a0b9b.npy
2,000453852fda,0,./data/train/0/000453852fda.npy
3,00047dfc96a9,0,./data/train/0/00047dfc96a9.npy
4,0004be0baf70,0,./data/train/0/0004be0baf70.npy


In [6]:
#!g1.1
class ClassificationDataset:
    
    def __init__(self, img_paths, targets): 
        self.img_paths = img_paths
        self.targets = targets

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, item):      
        img = np.load(self.img_paths[item]).astype(np.float32)
        targets = self.targets[item]
                
        return {
            'image': torch.tensor(img, dtype=torch.float),
            'targets': torch.tensor(targets, dtype=torch.float),
        }

In [7]:
#!g1.1
class EffNet(nn.Module):
    
    def __init__(self, params, out_dim):
        super(EffNet, self).__init__()
        self.enet = enet.EfficientNet.from_pretrained(params['backbone'])
        nc = self.enet._fc.in_features
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(
            in_channels=6, 
            out_channels=3, 
            kernel_size=3, 
            stride=1, 
            padding=3, 
            bias=False
        )
        #self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.myfc = nn.Sequential(
            nn.Dropout(params['dropout']),
            nn.Linear(nc, int(nc / 4)),
            nn.Dropout(params['dropout']),
            nn.Linear(int(nc / 4), out_dim)
        )
    
    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        return x

In [8]:
#!g1.1
def train(data_loader, model, optimizer, device):
    model.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='training'):
        inputs = data['image']
        targets = data['targets']
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()
        
def evaluate(data_loader, model, device):
    model.eval()
    val_targets = []
    val_outputs = []
    with torch.no_grad():
        for data in tqdm(data_loader, position=0, leave=True, desc='evaluating'):
            inputs = data['image']
            targets = data['targets']
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            output = model(inputs)
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            val_targets.extend(targets)
            val_outputs.extend(output)
    return val_outputs, val_targets

In [9]:
skf = StratifiedKFold(PARAMS['folds'], shuffle=True, random_state=PARAMS['seed'])
df['fold'] = -1
for i, (train_idxs, val_idxs) in enumerate(skf.split(df, df['target'])):
    df.loc[val_idxs, 'fold'] = i
display(df.head())

,id,target,img_path,fold
0,00034abb3629,0,./data/train/0/00034abb3629.npy,3
1,0004300a0b9b,0,./data/train/0/0004300a0b9b.npy,2
2,000453852fda,0,./data/train/0/000453852fda.npy,2
3,00047dfc96a9,0,./data/train/0/00047dfc96a9.npy,0
4,0004be0baf70,0,./data/train/0/0004be0baf70.npy,0


In [10]:
#!g1.1
pred, target = [], []
preds_val, target_val = [], []

if DEBUG:
    n_folds_train = 2
else:
    n_folds_train = PARAMS['folds'] if not PARAMS['folds_train'] else PARAMS['folds_train']
start_folds_train = 0

for fold_num in range(start_folds_train, n_folds_train):
    print('=' * 20, 'FOLD:', fold_num, '=' * 20)
    train_idxs = np.where((df['fold'] != fold_num))[0]
    val_idxs = np.where((df['fold'] == fold_num))[0]
    train_imgs, val_imgs = df.loc[train_idxs, 'img_path'].values, df.loc[val_idxs, 'img_path'].values
    train_targets, val_targets = df.loc[train_idxs, 'target'].values, df.loc[val_idxs, 'target'].values
    train_dataset = ClassificationDataset(img_paths=train_imgs, targets=train_targets)
    val_dataset = ClassificationDataset(img_paths=val_imgs, targets=val_targets)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, 
        batch_size=PARAMS['batch_size'],
        shuffle=True, 
        num_workers=PARAMS['workers']
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, 
        batch_size=PARAMS['batch_size'],
        shuffle=False, 
        num_workers=PARAMS['workers']
    )
    model = EffNet(params=PARAMS, out_dim=1)
    model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=PARAMS['lr'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, PARAMS['epochs'])
    print('train len:', len(train_dataset),'| val len:', len(val_dataset))
    best_file = '{}/model_best_{}.pth'.format(MDLS_PATH, fold_num)
    roc_auc_max = 0
    epochs_no_improve = 0
    for epoch in tqdm(range(PARAMS['epochs']), desc='epochs'):
        print(time.ctime(), 'epoch:', epoch)
        train(train_loader, model, optimizer, device=DEVICE)
        preds, val_targets = evaluate(val_loader, model, device=DEVICE)
        roc_auc = metrics.roc_auc_score(val_targets, preds)
        scheduler.step() #scheduler.step(epoch)
        content = '{} epoch {}, lr: {:.8f}, ROC-AUC: {:.4f}'.format(
            time.ctime(),
            epoch, 
            optimizer.param_groups[0]['lr'], 
            roc_auc
        )
        print(content)
        with open('{}/log_{}.txt'.format(MDLS_PATH, fold_num), 'a') as appender:
            appender.write(content + '\n')
        if roc_auc > roc_auc_max:
            torch.save(model.state_dict(), best_file)
            print('ROC-AUC improved {:.4f} --> {:.4f} model saved'.format(roc_auc_max, roc_auc))
            roc_auc_max = roc_auc
            preds_best, target_best = [], []
            with torch.no_grad():
                for data in tqdm(val_loader, desc='predict'):
                    img_data = data['image']
                    img_targets = data['targets']
                    img_data = img_data.to(DEVICE)
                    preds = model(img_data).sigmoid().cpu().numpy()
                    preds_best.extend(preds)
                    target_best.extend(img_targets.cpu().numpy())
            epochs_no_improve = 0
            print('val preds done:', len(preds_best), len(target_best))
        else:
            epochs_no_improve += 1
        if epochs_no_improve >= PARAMS['patience']:
            print('no improve for', epochs_no_improve, 'epochs | early stopping')
            early_stop = True
            break
    preds_val.extend(np.squeeze(preds_best))
    target_val.extend(np.squeeze(target_best))
    torch.save(
        model.state_dict(), 
        os.path.join('{}/model_final_{}.pth'.format(MDLS_PATH, fold_num))
    )
    del model, train_dataset, val_dataset, train_loader, val_loader
    torch.cuda.empty_cache()
    gc.collect()

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

==================== FOLD: 0 ====================
Loaded pretrained weights for efficientnet-b3
train len: 40132 | val len: 10033


Sat May 29 20:53:06 2021 epoch: 0



Sat May 29 21:08:54 2021 epoch 0, lr: 0.00099975, ROC-AUC: 0.9487
ROC-AUC improved 0.0000 --> 0.9487 model saved



val preds done: 10033 10033
Sat May 29 21:09:36 2021 epoch: 1



Sat May 29 21:19:15 2021 epoch 1, lr: 0.00099901, ROC-AUC: 0.9622
ROC-AUC improved 0.9487 --> 0.9622 model saved


...


Sun May 30 07:32:26 2021 epoch 10, lr: 0.00097044, ROC-AUC: 0.9641
Sun May 30 07:32:26 2021 epoch: 11



Sun May 30 07:42:05 2021 epoch 11, lr: 0.00096489, ROC-AUC: 0.9651
Sun May 30 07:42:05 2021 epoch: 12



Sun May 30 07:51:44 2021 epoch 12, lr: 0.00095888, ROC-AUC: 0.9602
Sun May 30 07:51:44 2021 epoch: 13



Sun May 30 08:01:23 2021 epoch 13, lr: 0.00095241, ROC-AUC: 0.9595
Sun May 30 08:01:23 2021 epoch: 14



Sun May 30 08:11:02 2021 epoch 14, lr: 0.00094550, ROC-AUC: 0.9584
no improve for 8 epochs | early stopping

time elapsed: 677 min 59 sec


In [11]:
roc_auc_max = 0
for th in np.linspace(.1, 1, 100):
    try:
        roc_auc = metrics.roc_auc_score(np.array(preds_val) > th, target_val)
        if roc_auc > roc_auc_max:
            roc_auc_max = roc_auc
            th_max = th
    except:
        pass
print('ROC-AUC max:', np.round(roc_auc_max, 2), '| th max:', np.round(th_max, 2))
with open(f'{MDLS_PATH}/th.json', 'w') as file:
    json.dump({
        'ROC-AUC max': roc_auc_max, 
        'th max': th_max,
    }, file)

ROC-AUC max: 0.98 | th max: 0.94
